<a href="https://colab.research.google.com/github/danrosher/Algorithms/blob/master/FineTuneSemanticSearchSymmetric.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -qq install numpy datasets sentence_transformers transformers pandas

In [ ]:

import numpy as np
from datasets import load_dataset

from sentence_transformers import SentenceTransformer, util
from transformers import pipeline

from random import sample, seed, shuffle
from sentence_transformers import InputExample, losses, evaluation
from torch.utils.data import DataLoader
import pandas as pd

In [ ]:
#model_name='sentence-transformers/all-MiniLM-L6-v2'
model_name='msmarco-MiniLM-L12-cos-v5'
sbert_model = SentenceTransformer(model_name)
#sbert_model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')
#sbert_model = SentenceTransformer('multi-qa-mpnet-base-cos-v1')
sbert_model.max_seq_length = 256



In [ ]:
from google.colab import drive

drive.mount('/g')

Drive already mounted at /g; to attempt to forcibly remount, call drive.mount("/g", force_remount=True).


In [ ]:
df = pd.read_csv('/g/MyDrive/semantic_search/data.csv')


In [ ]:
df.head()

,k,title,description,label
0,warehouse operative,Warehouse Operative (Nights),New4all Recruitment are hiring on behalf of a ...,1
1,warehouse operative,ELECTRICIAN,We require a qualified Electrician to work on ...,0
2,warehouse operative,Warehouse Operative,Red Recruitment Group urgently require a numbe...,1
3,warehouse operative,Class 1 HGV Training & Placement Programme,Do you enjoy driving?\nâ¢ Would you like to w...,0
4,warehouse operative,Factory Operative,"Are you hard working, reliable and looking for...",1


In [ ]:
def convert_data(df):
  return list(df.apply(lambda x: InputExample(texts = (x['k'], x['description'][:800]),label = float(x['label'])), axis = 1)) 

In [ ]:
good_training_data = df[df['label']==1].sample(n=2000)
bad_training_data = df[df['label']==0].sample(n=2000)

In [ ]:
#training_data = df.sample(frac=1) #return all in random order
sampled_training_data = convert_data(pd.concat([good_training_data,bad_training_data]).sample(frac=1))
len(sampled_training_data)

4000

In [ ]:
sampled_training_data[0].__dict__

{'guid': '',
 'texts': ('rail way',
  "Computer Vision Engineer â\x80\x93 Semi remote - London - Â£80k\nHexwired Recruitment is currently hiring for a rapidly expanding R&D company developing bespoke Simulation solutions in London. They're now seeking a Computer Vision Engineer with a solid understanding of CNN applications to be used in a range of novel Defence applications.\nThe company are working with a range of diverse customers so this is an excellent opportunity for someone seeking a challenging and rewarding role in data science. This a senior role within the business and there is scope to grow as the company expands!\nDue to the nature of the work, the majority of the work will be onsite and the company are willing to pay for the security clearance process.\nComputer Vision Engineer â\x80\x93 Semi remote - London - Â£80k\nKey Skills:\nM"),
 'label': 0.0}

In [ ]:
# Define the train dataset, the dataloader and the train loss
train_dataloader = DataLoader(sampled_training_data, shuffle=True, batch_size=64)

train_loss = losses.CosineSimilarityLoss(sbert_model)

In [ ]:
training_index = int(.8 * len(sampled_training_data))  # Get an 80/20 train/test split
train_examples = sampled_training_data[:training_index]


In [ ]:
# Evaluation data
evaluator = evaluation.EmbeddingSimilarityEvaluator.from_input_examples(sampled_training_data[training_index:], main_similarity=evaluation.SimilarityFunction.COSINE)
sbert_model.evaluate(evaluator)

0.6392403727814074

In [ ]:
# Tune the model
sbert_model.fit(
    train_objectives=[(train_dataloader, train_loss)], 
    output_path='/g/MyDrive/semantic_search/model',
    save_best_model=True,
    epochs=2,
#     warmup_steps=len(sampled_training_data) // 5, 
    evaluator=evaluator, 
    evaluation_steps=100
)

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/63 [00:00<?, ?it/s]

Iteration:   0%|          | 0/63 [00:00<?, ?it/s]

In [ ]:
evaluator(sbert_model,output_path='/g/MyDrive/semantic_search')

0.71670435853651

In [ ]:
sbert_model.save('/g/MyDrive/semantic_search/results')
